In [20]:
import google.generativeai as genai
import os
from getpass import getpass
import re
import time

In [22]:

# Prompt for the API key securely (input will be hidden)
api_key = getpass("Please enter your Google Gemini API Key: ")

# Prompt for the directory to save the file
# Example: 'generated_prompts' or 'data/sadness_prompts'
save_directory = '/workspace/MATS-research/data/emotion_user_prompts/'

# Define the full path for the output file
file_path = os.path.join(save_directory, "happiness.txt")

In [ ]:
#sadness prompt
#Generate {PROMPTS_PER_CALL} user prompts to an AI assistant where it is clear that the user is sad. Be creative on the topics of the prompts. Do not include a preface, numbering, or quotation marks. Each prompt must be on a new line.

#happiness prompt
#Generate {PROMPTS_PER_CALL} user prompts to an AI assistant where it is clear that the user is happy or content. Be creative on the topics of the prompts. Do not include a preface, numbering, or quotation marks. Each prompt must be on a new line.

In [23]:
# --- Configuration for Batch Generation ---
TOTAL_PROMPTS_NEEDED = 500
PROMPTS_PER_CALL = 50
NUM_CALLS = TOTAL_PROMPTS_NEEDED // PROMPTS_PER_CALL

# --- API Call and File Generation in a Loop ---

def clean_gemini_output(raw_text: str) -> str:
    """A robust function to clean raw model output and extract only the prompts."""
    lines = raw_text.strip().split('\n')
    cleaned_prompts = []
    for line in lines:
        line = line.strip()
        if not line or "here are" in line.lower() or "user prompts" in line.lower():
            continue
        line = re.sub(r'^\d+[\.\)]\s*', '', line)
        line = line.strip('"\'')
        if line:
             cleaned_prompts.append(line)
    return "\n".join(cleaned_prompts)

# List to store all generated prompts
all_generated_prompts = []

try:
    genai.configure(api_key=api_key)
    # Set a higher temperature for more varied outputs
    generation_config = { "temperature": 0.9, "top_p": 1.0 }
    
    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash-latest",
        generation_config=generation_config
    )

    # This prompt is now the same for every call
    prompt_template = f"""Generate {PROMPTS_PER_CALL} user prompts to an AI assistant where it is clear that the user is happy or content. Be creative on the topics of the prompts. Do not include a preface, numbering, or quotation marks. Each prompt must be on a new line."""

    print(f"Preparing to make {NUM_CALLS} API calls to generate {TOTAL_PROMPTS_NEEDED} prompts...")

    for i in range(NUM_CALLS):
        print(f"Making API call {i+1}/{NUM_CALLS}...")
        
        # Make the API call
        response = model.generate_content(prompt_template)
        
        # Clean and add the new prompts to our master list
        cleaned_text = clean_gemini_output(response.text)
        all_generated_prompts.extend(cleaned_text.split('\n'))
        
        # A small delay to be kind to the API
        time.sleep(1)

    # --- Save the final result ---
    # We remove any potential duplicates that might have been generated
    unique_prompts = list(dict.fromkeys(all_generated_prompts))
    final_output = "\n".join(unique_prompts)
    
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(final_output)
        
    print(f"\nSuccessfully generated and saved {len(unique_prompts)} unique prompts to:\n{file_path}")

except Exception as e:
    print(f"\n--- An Error Occurred ---")
    print(f"Error during call {i+1}: {e}")
    if all_generated_prompts:
        unique_prompts = list(dict.fromkeys(all_generated_prompts))
        final_output = "\n".join(unique_prompts)
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(final_output)
        print(f"Saved the {len(unique_prompts)} prompts generated so far to the file.")

Preparing to make 10 API calls to generate 500 prompts...
Making API call 1/10...


Making API call 2/10...
Making API call 3/10...
Making API call 4/10...
Making API call 5/10...
Making API call 6/10...
Making API call 7/10...
Making API call 8/10...
Making API call 9/10...
Making API call 10/10...

Successfully generated and saved 505 unique prompts to:
/workspace/MATS-research/data/emotion_user_prompts/happiness.txt
